In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#function for scale down the sensor output (0-4096 to 0-100)
def scaledown(df):
    originalRangeMax = 4096
    presentRangeMax = 100
    col = 0
    while(col < 8):
        df.update(df['flapper' + str(col)] / originalRangeMax * presentRangeMax)
        col += 1

In [3]:
colName_20 = ['time', \
                    'flapper0', 'flapper1', 'flapper2', 'flapper3', 'flapper4', 'flapper5', 'flapper6', 'flapper7', \
                    'pressure', 'light', \
                    'orientationX', 'orientationY', 'orientationZ', \
                    'accX', 'accY', 'accZ', \
                    'compassX', 'compassY', 'compassZ']

colName_26 = [
                    'time', \
                    'flapper0', 'flapper1', 'flapper2', 'flapper3', 'flapper4', 'flapper5', 'flapper6', 'flapper7', \
                    'pressure', 'light', \
                    'accX', 'accY', 'accZ', \
                    'compassX', 'compassY', 'compassZ', \
                    'rotationalSpeedX', 'rotationalSpeedY', 'rotationalSpeedZ', \
                    'quaternion0', 'quaternion1', 'quaternion2', 'quaternion3']
colName_23 = [
                    'time', \
                    'flapper0', 'flapper1', 'flapper2', 'flapper3', 'flapper4', 'flapper5', 'flapper6', 'flapper7', \
                    'bend', 'light', \
                    'accX', 'accY', 'accZ', \
                    'compassX', 'compassY', 'compassZ', \
                    'rotationalSpeedX', 'rotationalSpeedY', 'rotationalSpeedZ', \
                    'orientationX', 'orientationY', 'orientationZ']

In [4]:
def createDBFromCSV(filename, dbname, cols=20):
    #20 cols or 26cols
    colName = colName_20
    if cols == 26:
        colName = colName_26
    if cols == 23:
        colName = colName_23
    #read csv
    df = pd.read_csv(filename, names=colName)
    #drop '/xxx.csv' & 'finish'
    df.drop(df.head(1).index,inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    df = df.reset_index(drop=True)
    #
#     c = 0
#     while c < 8:
#         for i in range(len(df.index)):
#             df['flapper' + str(c)][i] = int(df['flapper' + str(c)][i])
#         c += 1
    #
    #scale down
    scaledown(df)
    #guarantee 'time' type
    df['time'] = pd.to_numeric(df['time'])
    #create database
    csv_database = create_engine('sqlite:///' + str(dbname) + '.db')
    tablename = 'sensorProcessed'
    df.to_sql(tablename, csv_database, if_exists='append')

In [20]:
run = 5
filename = '/Users/meng/Downloads/Dat01.csv' #file path
createDBFromCSV(filename, 'KFtest04', 23) # filename & num of cols